In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [2]:
import pandas as pd
import numpy as np

import time
import tqdm 
from datetime import datetime

import pickle
import torch
import torch.nn as nn

import shap as shap
#from captum.attr import KernelShap
from captum.attr import IntegratedGradients, Saliency, NoiseTunnel, InputXGradient
from lime import lime_tabular


# load data

In [3]:
train = pd.read_csv('data/german-train.csv')
test = pd.read_csv('data/german-test.csv')

In [4]:
#load data
data_train = train
data_test =test

display(data_train.head())

print('data, train:', data_train.shape)
print('data, test:', data_test.shape)

#split into X, y
X_train = data_train.loc[:, data_train.columns != 'credit-risk']
y_train = data_train['credit-risk']

X_test = data_test.loc[:, data_test.columns != 'credit-risk']
y_test = data_test['credit-risk']


X=X_train
y=y_train
print('----- TRAIN -----')
print('X, shape:', X.shape)
print('y, shape:', y.shape)
print('#class1: ', sum(y), f', prop = {sum(y)/len(y)}')
print('#class0:', sum(y==0), f', prop = {sum(y==0)/len(y)}')

X=X_test
y=y_test
print('----- TEST -----')
print('X, shape:', X.shape)
print('y, shape:', y.shape)
print('#class1: ', sum(y), f', prop = {sum(y)/len(y)}')
print('#class0:', sum(y==0), f', prop = {sum(y==0)/len(y)}')


,status,duration,credit-history,purpose,amount,savings,employment-duration,installment-rate,personal-status-sex,other-debtors,...,property,age,other-installment-plans,housing,number-credits,job,people-liable,telephone,foreign-worker,credit-risk
0,1,15,3,4,1213,4,5,4,5,1,...,3,47,2,3,1,3,1,2,1,1
1,2,24,3,4,2439,2,2,4,2,1,...,4,35,3,3,1,3,1,2,1,0
2,1,60,3,1,10366,2,5,2,5,1,...,3,42,3,3,1,4,1,2,1,1
3,4,6,1,7,1047,2,3,2,2,1,...,3,50,3,3,1,2,1,1,1,1
4,2,18,3,4,1882,2,3,4,2,1,...,2,25,1,2,2,3,1,1,1,0


data, train: (800, 21)
data, test: (200, 21)
----- TRAIN -----
X, shape: (800, 20)
y, shape: (800,)
#class1:  560 , prop = 0.7
#class0: 240 , prop = 0.3
----- TEST -----
X, shape: (200, 20)
y, shape: (200,)
#class1:  140 , prop = 0.7
#class0: 60 , prop = 0.3


# load models

In [5]:
#load models

#####logistic regression
model_filename = 'models/model_logistic.pkl'
model_logistic = pickle.load(open(model_filename, 'rb'))

######gradient boosted tree
model_filename = 'models/model_gb.pkl'
model_gb = pickle.load(open(model_filename, 'rb'))

######random forest
model_filename = 'models/model_rf.pkl'
model_rf = pickle.load(open(model_filename, 'rb'))

######FFNN

#module
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, seed=12345):
        super().__init__()
        
        torch.manual_seed(seed)
        
        #variables
        self.input_size = input_size
        self.hidden_size = hidden_size
        #layers architecture
        self.linear_layer1 = nn.Linear(self.input_size, self.hidden_size)
        self.linear_layer2 = nn.Linear(self.hidden_size, self.hidden_size*2)
        self.linear_layer3 = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.linear_layer4 = nn.Linear(self.hidden_size, 1)
        
    def forward(self, inputs):
        out = self.linear_layer1(inputs)
        out = nn.functional.relu(out)
        out = self.linear_layer2(out)
        out = nn.functional.relu(out)
        out = self.linear_layer3(out)
        out = nn.functional.relu(out)
        out = self.linear_layer4(out)
        out = torch.sigmoid(out)
        return out
    
    def predict_proba(self, X):
        X = torch.FloatTensor(X)
        class1_probs = self.forward(X).detach().numpy()
        class0_probs = 1-class1_probs
        return np.hstack((class0_probs, class1_probs))
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


model_filename = 'models/model_nn.pkl'
model_nn = torch.load(model_filename)


##### NN logistic regression

#module

#create model class
class LogisticRegressionNN(nn.Module):
    def __init__(self, input_size, seed=12345):
        super().__init__()
        
        torch.manual_seed(seed)
        
        #variables
        self.input_size = input_size
        #layers
        self.linear_layer = nn.Linear(self.input_size, 1)
        
    def forward(self, inputs):
        out = self.linear_layer(inputs)
        out = torch.sigmoid(out)
        return out
    
    def predict_proba(self, X):
        X = torch.tensor(X).type(torch.FloatTensor)
        class1_probs = self.forward(X).detach().numpy()
        class0_probs = 1-class1_probs
        return np.hstack((class0_probs, class1_probs))
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


model_filename = 'models/model_nn_logistic.pkl'
model_nn_logistic = torch.load(model_filename)

# ---------- METHOD 1: kernelshap ----------

In [36]:
#kernelshap function
def explain_kernelshap(predict_fn, instances, filename,
                       background_data, nsamples):
    #run kernelshap
    explainer = shap.KernelExplainer(model=predict_fn, data=background_data)
    shap_values = explainer.shap_values(X=instances, nsamples=nsamples)
    
    #save shap values
    pickle.dump(shap_values, open(filename, 'wb'))

    return shap_values


In [37]:
#predict functions --> input for explain_kernelshap

# predict_fn: predicts probability of class1
#     in: instances, 2D np.array, n(=#datapoints) x p(=#features)
#     out: predictions, 1D np.array, n


def predict_fn_logistic(instances):
    return model_logistic.predict_proba(instances)[:, 1]

def predict_fn_gb(instances):
    return model_gb.predict_proba(instances)[:, 1]

def predict_fn_rf(instances):
    return model_rf.predict_proba(instances)[:, 1]

def predict_fn_nn(instances):
    return model_nn.predict_proba(instances)[:, 1]

def predict_fn_nn_logistic(instances):
    return model_nn_logistic.predict_proba(instances)[:, 1]


In [40]:
#number instances to explain
n= X_test.shape[0] #10

#general arguments
instances = X_test.values[0:n, :]

#shap arguments
background_data = X_test.values
nsamples = 2**7


model_names = ['logistic', 'gb', 'rf', 'nn', 'nn_logistic']
predict_fns = {'logistic': predict_fn_logistic, 
               'gb': predict_fn_gb, 
               'rf': predict_fn_rf, 
               'nn': predict_fn_nn,
               'nn_logistic': predict_fn_nn_logistic}
filenames_ks = {'logistic': 'explanations/expl_kernelshap_logistic.pkl', 
                'gb': 'explanations/expl_kernelshap_gb.pkl', 
                'rf': 'explanations/expl_kernelshap_rf.pkl', 
                'nn': 'explanations/expl_kernelshap_nn.pkl',
                'nn_logistic': 'explanations/expl_kernelshap_nn_logistic.pkl'}

In [41]:
#explain all models using kernelshap
expl_shap = {m: explain_kernelshap(predict_fns[m], instances, filenames_ks[m], background_data, nsamples) 
             for m in model_names}

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/200 [00:00<?, ?it/s]

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/200 [00:00<?, ?it/s]

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/200 [00:00<?, ?it/s]

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/200 [00:00<?, ?it/s]

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
#load kernelshap explanations

model_names = ['logistic', 'gb', 'rf', 'nn', 'nn_logistic']

filenames_ks = {'logistic': 'explanations/expl_kernelshap_logistic.pkl', 
                'gb': 'explanations/expl_kernelshap_gb.pkl', 
                'rf': 'explanations/expl_kernelshap_rf.pkl', 
                'nn': 'explanations/expl_kernelshap_nn.pkl',
                'nn_logistic': 'explanations/expl_kernelshap_nn_logistic.pkl'}


expl_shap = {m: pickle.load(open(filenames_ks[m], 'rb')) for m in model_names}

# ---------- METHOD 2: vanilla gradient ----------

In [42]:
def explain_vanilla_grad(model, instances, filename):
    model.zero_grad()
    method = Saliency(model)
    attr = method.attribute(instances)
    
    attr_np = attr.numpy()
    pickle.dump(attr_np, open(filename, 'wb'))
    return attr_np

## NN

In [43]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn
filename = 'explanations/expl_vanillagrad_nn.pkl'

#run explanation 
expl_vanillagrad_nn = explain_vanilla_grad(model, instances, filename)

#load explanation
expl_vanillagrad_nn = pickle.load(open(filename, 'rb'))

Input Tensor 0 did not already require gradients, required_grads has been set automatically.


## logistic

In [44]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn_logistic
filename = 'explanations/expl_vanillagrad_nn_logistic.pkl'

#run explanation 
expl_vanillagrad_nn_logistic = explain_vanilla_grad(model, instances, filename)

#load explanation
expl_vanillagrad_nn_logistic = pickle.load(open(filename, 'rb'))

# ---------- METHOD 3: GRADIENT*INPUT ----------

In [45]:
def explain_gradtinput(model, instances, filename):
    model.zero_grad()
    method = InputXGradient(model)
    attr = method.attribute(instances)
    
    attr_np = attr.detach().numpy()
    pickle.dump(attr_np, open(filename, 'wb'))
    return attr_np

## NN

In [46]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn
filename = 'explanations/expl_gradtinput_nn.pkl'

#run explanation
expl_gradtinput_nn = explain_gradtinput(model, instances, filename)

#load explanation
expl_gradtinput_nn = pickle.load(open(filename, 'rb'))

## logistic

In [47]:
#parameters
n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

model = model_nn_logistic
filename = 'explanations/expl_gradtinput_nn_logistic.pkl'

#run explanation
expl_gradtinput_nn_logistic = explain_gradtinput(model, instances, filename)

#load explanation
expl_gradtinput_nn_logistic = pickle.load(open(filename, 'rb'))

# ---------- METHOD 4: integrated gradients ----------

In [48]:
def explain_integrated_grad(model, instances, n_steps=50):
    model.zero_grad()
    method = IntegratedGradients(model)
    attr = method.attribute(instances, n_steps=n_steps)
    return attr.numpy()

## explore convergence, NN

In [49]:
def check_convergence(model, instances, nsamples_list, filename, expl_method=['integratedgrad', 'smoothgrad']):
    #dict to store attributions for each sample_size in nsamples_list (sample_size: attributions)
    convergence_attr = {}
    
    for i in nsamples_list:
        
        print(f'nsamples={i}')
        start = time.time()
        print(f'   start: {datetime.now()}')
        
        #run explanation method
        if expl_method=='integratedgrad':
            expl_i = explain_integrated_grad(model, instances, n_steps=i)
        elif expl_method=='smoothgrad':
            expl_i = explain_smoothgrad(model, instances, nt_samples=i)
        #store values
        convergence_attr[i] = expl_i
        
        stop = time.time()
        print(f'   stop: {datetime.now()}')
        print(f'   duration: {(stop-start)/60} min')
        
    #save data
    pickle.dump(convergence_attr, open(filename, 'wb'))

    return convergence_attr
    

In [51]:
#explore convergence, nn --- try different n_steps

#parameters
model = model_nn

n = X_test.shape[0] #10
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='integratedgrad'
filename=f'convergence/convergence_{expl_method}_nn.pkl'

#check convergence
convergence_integratedgrad_nn = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_integratedgrad_nn = pickle.load(open(filename, 'rb'))


nsamples=50
   start: 2021-12-14 22:28:09.066224
   stop: 2021-12-14 22:28:09.244316
   duration: 0.002968287467956543 min
nsamples=100
   start: 2021-12-14 22:28:09.244603
   stop: 2021-12-14 22:28:09.690815
   duration: 0.007436811923980713 min
nsamples=200
   start: 2021-12-14 22:28:09.690945
   stop: 2021-12-14 22:28:11.162305
   duration: 0.024522634347279866 min
nsamples=400
   start: 2021-12-14 22:28:11.162429
   stop: 2021-12-14 22:28:18.162519
   duration: 0.11666810115178426 min
nsamples=600
   start: 2021-12-14 22:28:18.162664
   stop: 2021-12-14 22:28:39.869158
   duration: 0.3617744525273641 min
nsamples=800
   start: 2021-12-14 22:28:39.869667
   stop: 2021-12-14 22:29:24.842118
   duration: 0.7495404322942097 min
nsamples=1000
   start: 2021-12-14 22:29:24.842697
   stop: 2021-12-14 22:30:39.649159
   duration: 1.246773934364319 min
nsamples=1500
   start: 2021-12-14 22:30:39.649552
   stop: 2021-12-14 22:33:21.984269
   duration: 2.705578104654948 min


In [52]:
#explore convergence, nn --- plot


In [53]:
#explore convergence, nn --- pick and save explanation


## explore convergence, logistic

In [54]:
#explore convergence, logistic

#explore convergence, nn --- try different n_steps

#parameters
model = model_nn_logistic

n = X_test.shape[0] #10
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='integratedgrad'
filename=f'convergence/convergence_{expl_method}_nn_logistic.pkl'



#check convergence
convergence_integratedgrad_nn_logistic = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_integratedgrad_nn_logistic = pickle.load(open(filename, 'rb'))



nsamples=50
   start: 2021-12-14 22:33:22.011389
   stop: 2021-12-14 22:33:22.134502
   duration: 0.002051834265391032 min
nsamples=100
   start: 2021-12-14 22:33:22.134664
   stop: 2021-12-14 22:33:22.529890
   duration: 0.0065870523452758786 min
nsamples=200
   start: 2021-12-14 22:33:22.530025
   stop: 2021-12-14 22:33:24.018411
   duration: 0.02480638027191162 min
nsamples=400
   start: 2021-12-14 22:33:24.018562
   stop: 2021-12-14 22:33:30.820982
   duration: 0.113373597462972 min
nsamples=600
   start: 2021-12-14 22:33:30.821115
   stop: 2021-12-14 22:33:51.497825
   duration: 0.3446115175882975 min
nsamples=800
   start: 2021-12-14 22:33:51.498141
   stop: 2021-12-14 22:34:34.977672
   duration: 0.7246584057807922 min
nsamples=1000
   start: 2021-12-14 22:34:34.977992
   stop: 2021-12-14 22:35:44.845328
   duration: 1.1644553542137146 min
nsamples=1500
   start: 2021-12-14 22:35:44.845652
   stop: 2021-12-14 22:38:20.423550
   duration: 2.592964434623718 min


# ---------- METHOD 5: smoothgrad ----------

In [55]:
def explain_smoothgrad(model, instances, nt_samples=5):
    model.zero_grad()
    method = NoiseTunnel(Saliency(model))
    attr = method.attribute(instances, nt_type='smoothgrad', nt_samples=nt_samples)
    return attr.numpy()

## NN

In [56]:
#smoothgrad
#explore convergence, nn --- try different n_steps

#parameters
model = model_nn

n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='smoothgrad'
filename=f'convergence/convergence_{expl_method}_nn.pkl'

#check convergence
convergence_smoothgrad_nn = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_smoothgrad_nn = pickle.load(open(filename, 'rb'))


nsamples=50
   start: 2021-12-14 22:38:20.443763
   stop: 2021-12-14 22:38:20.640198
   duration: 0.0032739837964375815 min
nsamples=100
   start: 2021-12-14 22:38:20.640278
   stop: 2021-12-14 22:38:21.080281
   duration: 0.00733335018157959 min
nsamples=200
   start: 2021-12-14 22:38:21.080781
   stop: 2021-12-14 22:38:22.596869
   duration: 0.02526810169219971 min
nsamples=400
   start: 2021-12-14 22:38:22.597148
   stop: 2021-12-14 22:38:27.896745
   duration: 0.08832656542460124 min
nsamples=600
   start: 2021-12-14 22:38:27.896875
   stop: 2021-12-14 22:38:46.835533
   duration: 0.3156442681948344 min
nsamples=800
   start: 2021-12-14 22:38:46.835653
   stop: 2021-12-14 22:39:28.971945
   duration: 0.702271330356598 min
nsamples=1000
   start: 2021-12-14 22:39:28.972205
   stop: 2021-12-14 22:40:45.289795
   duration: 1.2719593445460002 min
nsamples=1500
   start: 2021-12-14 22:40:45.290240
   stop: 2021-12-14 22:43:34.562798
   duration: 2.8212087313334147 min


## logistic

In [57]:
#smoothgrad
#explore convergence, nn --- try different n_steps

#parameters
model = model_nn_logistic

n = X_test.shape[0]
instances = X_test.values[0:n, :]
instances = torch.FloatTensor(instances)

nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500]
expl_method='smoothgrad'
filename=f'convergence/convergence_{expl_method}_nn_logistic.pkl'

#check convergence
convergence_smoothgrad_nn_logistic = check_convergence(model, instances, nsamples_list, filename, expl_method) #!!!

#load file
convergence_smoothgrad_nn_logistic = pickle.load(open(filename, 'rb'))


nsamples=50
   start: 2021-12-14 22:43:34.625307
   stop: 2021-12-14 22:43:34.745562
   duration: 0.002004249890645345 min
nsamples=100
   start: 2021-12-14 22:43:34.745690
   stop: 2021-12-14 22:43:35.133826
   duration: 0.00646888017654419 min
nsamples=200
   start: 2021-12-14 22:43:35.137050
   stop: 2021-12-14 22:43:36.539881
   duration: 0.023380466302235923 min
nsamples=400
   start: 2021-12-14 22:43:36.540301
   stop: 2021-12-14 22:43:43.261272
   duration: 0.1120161493619283 min
nsamples=600
   start: 2021-12-14 22:43:43.261397
   stop: 2021-12-14 22:44:04.459182
   duration: 0.3532963832219442 min
nsamples=800
   start: 2021-12-14 22:44:04.459325
   stop: 2021-12-14 22:44:50.362570
   duration: 0.7650536696116129 min
nsamples=1000
   start: 2021-12-14 22:44:50.363019
   stop: 2021-12-14 22:46:08.352017
   duration: 1.299816115697225 min
nsamples=1500
   start: 2021-12-14 22:46:08.352683
   stop: 2021-12-14 22:48:54.670974
   duration: 2.7719712018966676 min


# ---------- METHOD 6: lime ----------

In [6]:
def explain_lime(predict_fn, instances, training_data, cat_idxs, n_samples, filename,
                 mode=['classification', 'regression'], seed=12345):
    #create lime explainer
    np.random.seed(seed)
    explainer = lime_tabular.LimeTabularExplainer(training_data=training_data, 
                                                  mode=mode, 
                                                  discretize_continuous=False,
                                                  sample_around_instance=True,
                                                  random_state=seed, 
                                                  categorical_features=cat_idxs)
    #explain each data point in 'instances'
    exps = []
    num_feat = instances.shape[1]
    for x in instances:
        exp = explainer.explain_instance(x, 
                                         predict_fn=predict_fn,
                                         num_samples=n_samples,
                                         num_features=num_feat).local_exp[1]
        #format explanations
        exp = sorted(exp, key=lambda tup: tup[0])
        exp = [t[1] for t in exp]
        exps.append(exp)
    
    #save explanations
    exps = np.array(exps)
    pickle.dump(exps, open(filename, 'wb'))
    
    return exps #n x p (same dimensions as 'instances')

In [7]:
def check_convergence_lime(predict_fn, instances, training_data, cat_idxs, filename_lime, mode,
                           nsamples_list, filename_convergence):
    #dict to store attributions for each sample_size in nsamples_list (sample_size: attributions)
    convergence_attr = {}
    
    for i in nsamples_list:
        
        print(f'nsamples={i}')
        start = time.time()
        print(f'   start: {datetime.now()}')
        
        #run explanation method
        expl_i = explain_lime(predict_fn, instances, training_data, cat_idxs, 
                              n_samples=i, filename=f'{filename_lime}_n{i}.pkl', mode=mode)
        #store values
        convergence_attr[i] = expl_i
        
        stop = time.time()
        print(f'   stop: {datetime.now()}')
        print(f'   duration: {(stop-start)/60} min')
        
    #save data
    pickle.dump(convergence_attr, open(filename_convergence, 'wb'))

    return convergence_attr
    

In [60]:
#dictionaries
model_names = ['logistic', 'gb', 'rf', 'nn', 'nn_logistic']
predict_fns_2classes = {'logistic': model_logistic.predict_proba, 
               'gb': model_gb.predict_proba, 
               'rf': model_rf.predict_proba, 
               'nn': model_nn.predict_proba,
               'nn_logistic': model_nn_logistic.predict_proba}


#arguments that are constant
n = X_test.shape[0] #10
instances = X_test.values[0:n, :]
training_data = X_train.values

cat_vars = ['two_year_recid', 'c_charge_degree_F', 'sex_Female', 'race']
cat_idxs = [list(X_train.columns).index(var) for var in cat_vars]

mode = 'classification'
nsamples_list = [50, 100, 200, 400, 600, 800, 1000, 1500, 2000, 2500, 3000]


ValueError: 'two_year_recid' is not in list

In [ ]:
#for each model
for m in range(0, len(model_names)):
    model = model_names[m]
    print(f'******convergence analysis for {model}******')
    
    #run convergence analysis
    filename_lime=f'explanations/expl_lime_{model}'
    filename_convergence=f'convergence/convergence_lime_{model}.pkl'
    _ = check_convergence_lime(predict_fn=predict_fns_2classes[model], 
                           instances=instances, 
                           training_data=training_data, 
                           cat_idxs=cat_idxs, 
                           filename_lime=filename_lime, 
                           mode=mode,
                           nsamples_list=nsamples_list, 
                           filename_convergence=filename_convergence)
    